In [ ]:
import sys
#sys.path.append('../../../pdl/pdl/utils/')

import numpy as np
import matplotlib.pyplot as plt
#from logs import enable_logging, logging 
from importlib import reload
import iterativeMethods as im


In [ ]:
#enable_logging(lvl=100)
A = np.array([[2, 1], [5, 7]])
f = np.array([11, 13])

u, res = im.jacobi(A, f)

For the real Jacobi method instead the spectral radius of the updating matrix is < 1.
For the real Jacobi methods it holds the fact that if the matrix is diagonally dominant then the spectral radius of the updating matrix is guaranteed to be < 1. (http://www.cs.unipr.it/~bagnara/Papers/PDF/SIREV95.pdf)
This condition does not hold for the paper Jacobi method.

In [ ]:
jacobi_eig = np.linalg.eigvals(np.linalg.inv(np.diag(np.diag(A))).dot(A-np.diag(np.diag(A))))
spectral_radius = np.max(np.abs(jacobi_eig))
print(spectral_radius)

Build the matrix A for the 2D Poisson problem

In [ ]:
N = 3
A = np.eye(N**2)
# Domani length
L = 1.0
# Cell size
h = L/(N-1)

# set homegenous dirichlet BC value
b = 1.0

#Initilize forcing term
f = np.ones(N**2)*b

for i in range(N, N**2-N):
    if (i%N != 0 and i%N != N-1):
        # Left and right neigh
        A[i][i-1] = -0.25 
        A[i][i+1] = -0.25
        # Up and low neigh
        A[i][i-N] = -0.25 
        A[i][i+N] = -0.25 
        # set forcing term
        f[i] = 0


Obtain the solution with jacobi method

In [ ]:
u, res = im.jacobi(A, f, max_iters=10000,tol = 1e-3)
#u = np.linalg.inv(A).dot(f)

Plot the solution
Nice reference for contour plots https://www.python-course.eu/matplotlib_contour_plot.php

In [ ]:
x = np.linspace(0, L, N)
y = np.linspace(0, L, N)
X, Y = np.meshgrid(x, y)

Z = np.reshape(u, [N, N])
plt.figure()
cp = plt.contourf(X, Y, Z)
plt.colorbar(cp)
plt.title('Filled Contours Plot')
plt.xlabel('x [-]')
plt.ylabel('y [-]')
plt.show()

Obtain the same solution with reset operator G


In [ ]:
N = 35
a = np.ones(N**2)
b = -np.ones(N**2-1)*0.25
c = -np.ones(N**2-N)*0.25

A = np.diag(a) + np.diag(b, 1) + np.diag(b, -1) + np.diag(c, N) + np.diag(c, -N)
#print(A)


b_top_idx = np.arange(N)
b_bottom_idx = np.arange(N**2-N, N**2)
b_left_idx = np.linspace(N, N**2-2*N, N-2, dtype = int)
b_right_idx = np.linspace(2*N-1, N**2-N-1, N-2, dtype = int)


print(b_top_idx)
print(b_bottom_idx)
print(b_left_idx)
print(b_right_idx)

b_idx = np.append(b_top_idx, b_bottom_idx)
b_idx = np.append(b_idx, b_left_idx)
b_idx = np.append(b_idx, b_right_idx)
#print(b_idx)
b = np.ones(np.shape(b_idx))*3.0
#print(b)
f = np.ones(N**2)

u, res = im.jacobi(A, f, b_idx = b_idx, b = b,max_iters=1000,tol = 1e-2)

x = np.linspace(0, L, N)
y = np.linspace(0, L, N)
X, Y = np.meshgrid(x, y)

Z = np.reshape(u, [N, N])
plt.figure()
cp = plt.contourf(X, Y, Z)
plt.colorbar(cp)
plt.title('Filled Contours Plot')
plt.xlabel('x [-]')
plt.ylabel('y [-]')
plt.show()



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# One convolution with a 3x3 kernel
#net = nn.Conv2d(1, 1, 3, padding=1, bias=False)

# Do we want to use 3 layers?
jac = nn.Conv2d(1, 1, 3, padding=1, bias=False)

initial_weights = torch.zeros(1,1,3,3)
initial_weights[0,0,0,1] = 0.25
initial_weights[0,0,2,1] = 0.25
initial_weights[0,0,1,0] = 0.25
initial_weights[0,0,1,2] = 0.25
jac.weight = nn.Parameter(initial_weights)

for param in jac.parameters():
    param.requires_grad = False
    
for name, param in jac.named_parameters():
    print(name, param)


    
net = nn.Sequential(
    nn.Conv2d(1, 1, 3, padding=1, bias=False),
    nn.Conv2d(1, 1, 3, padding=1, bias=False),
    nn.Conv2d(1, 1, 3, padding=1, bias=False),
)


# Initialize the wights s.t. H corresponds T (see 2.3.1)
#initial_weights = torch.rand(1,1,3,3)*0.01
#initial_weights[0,0,0,1] = 0.25
#initial_weights[0,0,2,1] = 0.25
#initial_weights[0,0,1,0] = 0.25
#initial_weights[0,0,1,2] = 0.25
#initial_weights[0,0,1,1] = 1.0
#net.weight = nn.Parameter(initial_weights)

# Set the optimizer, you have to play with lr: if too big nan
optim = torch.optim.SGD(net.parameters(), lr=1e-6)
#optim = torch.optim.Adam(net.parameters(), lr=1e-6)
#optim = torch.optim.ASGD(net.parameters())
# SGD is much faster

for name, param in net.named_parameters():
    print(name, param)



In [ ]:
# Build G matrix and B to reset the boundaries
G = np.eye(N**2)
G[b_idx, b_idx] -= 1
B = np.zeros(N**2)
B[b_idx] = b

# Convert G and B to torch
Bt = torch.zeros(1,1,N**2,1)
Bt[0, 0, :,0] = torch.from_numpy(B)
Gt = torch.zeros(1,1,N**2,N**2)
Gt[0,0,:, :] = torch.from_numpy(G)

# Build T matrix for standard jacobi iteration
I = torch.zeros(1,1,N**2,N**2)
I[0,0,:,:] = torch.eye(N**2,N**2)
At = torch.zeros(1,1,N**2,N**2)
At[0,0,:, :] = torch.from_numpy(A)
T = I - At


In [ ]:
losses = []


The output is obtained applying k times G(Tu+net(Tu)+f+net(f)-net(u)) + B = G(Tu+HTu+f+Hf-Hu) + B.
G is the I with zeros in the boundary nodes
B is a vector with the boundary values in the boundary nodes and zeros elsewhere.

In [ ]:
# Loop over problem istances(TOADD) and k 
for _ in range(10):
    net.zero_grad()
    # Sample k and u0
    #k = 300
    k = np.random.randint(1,20)
    u0 = torch.randn(1, 1, N**2, 1, requires_grad=True)
    
    b = np.random.rand(np.shape(b_idx)[0])*3.0
    B = np.zeros(N**2)
    B[b_idx] = b
    # Convert G and B to torch
    Bt = torch.zeros(1,1,N**2,1)
    Bt[0, 0, :,0] = torch.from_numpy(B)

    
    # Solve the same problem, at each iteration the only thing changing are the weights, which are optimized
    for _ in range(100):
        

        # Initialize f: we use a zero forcing term for training
        f = np.zeros(N**2)
        ft = torch.zeros(1, 1, N**2, 1)
        #ft[0,0,:,0] = torch.from_numpy(f)
        
        # Compute ustar = gtt ground truth solution torch TODO obtain ground_truth with LU factorization much faster
        ground_truth, _ = im.jacobi(A, f, b_idx = b_idx, b = b,max_iters=1000,tol = 1e-7)
        gtt = torch.zeros(1,1,N**2, 1)
        gtt[0, 0, :, 0] = torch.from_numpy(ground_truth)

        # Initialize output 
        output = Gt@(u0) + Bt
        
        # Obtain solution by iteratink k times, applyng the net is equivalent to pre-multiply by H
        for _ in range(k):
            # The two exression should be equivalent
            #output = Gt@(T@(output) + Gt@(net((Gt@(T@(output)+ft)+Bt).view(1,1,N, N) - output.view(1,1,N, N)).view(1, 1,N**2, 1)) + ft ) + Bt
            output = Gt@(T@(output) + ft) + Bt + Gt@(net((Gt@(T@(output)+ft)+Bt).view(1,1,N, N) - output.view(1,1,N, N)).view(1, 1,N**2, 1)) 
            #output = Gt@((jac(output).view(1,1,N, N)).view(1, 1,N**2, 1) + ft) + Bt + Gt@(net((Gt@(T@(output)+ft)+Bt).view(1,1,N, N) - output.view(1,1,N, N)).view(1, 1,N**2, 1)) 

        # Define the loss, CHECK if it is correct wrt paper
        loss = F.mse_loss(gtt, output)
        
        """
        factor = 1.0
        for name, param in net.named_parameters():
            loss += factor *(torch.sum(param))
        """

        
        if loss <= 1e-6:
            break
            
        # Backpropagation
        loss.backward(retain_graph=False)
        
        # SGD step
        optim.step()
        
        # Store lossses for visualization
        losses.append(loss.item())

for name, param in net.named_parameters():
    print(name, param)

Plot the losses

In [ ]:
import matplotlib.pyplot as plt
color_map = plt.get_cmap('cubehelix')
colors = color_map(np.linspace(0.1, 1, 10))

losses_fig = plt.figure()
n_iter = np.arange(np.shape(losses)[0])
plt.plot(n_iter, losses, color = colors[0], linewidth = 1, linestyle = "-", marker = "",  label='Loss')

plt.legend(bbox_to_anchor=(0., -0.3), loc=3, borderaxespad=0.)
plt.xlabel('n iteration', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.title('Loss')
plt.grid(True, which = "both", linewidth = 0.5,  linestyle = "--")

print("final loss is {0}".format(losses[-1]))
#losses_fig.savefig('gridSearch.eps', bbox_inches='tight')

In [ ]:
print(k)

      

Compare solution obtained with ground truth 

In [ ]:
k = 500
# Initialize u0
#a = np.random.rand(N**2)
output = torch.ones(1, 1, N**2, 1, requires_grad=False)
#output[0,0,:,0] = torch.from_numpy(a, requires_grad=True)
output = Gt@(output) + Bt

# Initialize f randomly
f = np.random.rand(N**2)
f = np.ones(N**2)
ft = torch.ones(1, 1, N**2, 1)
ft[0,0,:,0] = torch.from_numpy(f)

# Compute ustar = gtt ground truth solution torch
ground_truth, _ = im.jacobi(A, f, b_idx = b_idx, b = b,max_iters=10000,tol = 1e-7)
gtt = torch.zeros(1,1,N**2, 1)
gtt[0, 0, :, 0] = torch.from_numpy(ground_truth)

for _ in range(k):
    #output = Gt@(T@(output) + Gt@(net((Gt@(T@(output)+ft)+Bt).view(1,1,N, N) - output.view(1,1,N, N)).view(1, 1,N**2, 1)) + ft ) + Bt
    output = Gt@(T@(output) + ft) + Bt + Gt@(net((Gt@(T@(output)+ft)+Bt).view(1,1,N, N) - output.view(1,1,N, N)).view(1, 1,N**2, 1)) 


In [ ]:
print(F.mse_loss(gtt, output))

In [ ]:
T.size()

In [ ]:
Bt.size()

In [ ]:
ft.size()